In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings
import igraph

In [71]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Philadelphia, United States','Denver, United States','Ghent, Belgium','Amsterdam, Netherlands',
          'Dhaka Metropolitan, Bangladesh']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 0 # (default)

thresholds
# 'Dublin, Ireland' is problematic; road system is too dense, has separate script

[300, 600, 1000]

In [72]:
# Block 1 city boundaries

warnings.filterwarnings("ignore")
bound_df = pd.DataFrame()
for i in cities:
    bound_df = pd.concat([bound_df, pd.DataFrame(ox.geocoder.geocode_to_gdf(i))])
    
bound_df = bound_df.reset_index()
bound_df = bound_df.loc[:,bound_df.columns!='index']
bound_df = gpd.GeoDataFrame(bound_df, geometry = 'geometry', crs = 4326)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')
bound_df

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.002410,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797
1,"POLYGON ((-105.10988 39.62710, -105.10761 39.6...",39.914209,39.614315,-104.599700,-105.109885,327393456,relation,1411339,39.739236,-104.984862,"Denver, Colorado, United States",boundary,administrative,0.905476
2,"POLYGON ((3.57976 51.03027, 3.57992 51.03022, ...",51.188886,50.979542,3.849325,3.579762,282032351,relation,897671,51.084199,3.732851,"Ghent, Gent, East Flanders, Flanders, Belgium",boundary,administrative,0.855558
3,"MULTIPOLYGON (((4.72876 52.40071, 4.73371 52.4...",52.431064,52.278174,5.079162,4.728756,282057196,relation,271110,52.372760,4.893604,"Amsterdam, North Holland, Netherlands",boundary,administrative,0.946813
4,"POLYGON ((90.32978 23.75081, 90.32981 23.75050...",23.899531,23.668077,90.508832,90.329784,328693247,relation,13663697,23.783663,90.403246,"Dhaka Metropolitan, Dhaka District, Dhaka Divi...",boundary,administrative,0.710000


In [133]:
# Block 2 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()
#roads = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    road_nodes.append(road_node)
    #road = road_node.reset_index()
    #roads.append(road)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Philadelphia done 2.36 mns
Denver done 5.65 mns
Ghent done 6.4 mns
Amsterdam done 8.03 mns
Dhaka Metropolitan done 8.88 mns


,from,to,key,osmid,name,highway,oneway,length,geometry,ref,lanes,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m
0,103237949,103353127,103237949-103353127,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617..."
1,103237949,103353090,103237949-103353090,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364..."
2,103237949,103237976,103237949-103237976,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552..."
3,103237976,103238007,103237976-103238007,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797..."
4,103237976,103590312,103237976-103590312,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,9726316545-5550941352,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935..."
338098,9726316545,9726316548,9726316545-9726316548,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962..."
338099,9726316548,9726316545,9726316548-9726316545,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998..."
338100,9728655691,110122187,9728655691-110122187,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869..."


In [134]:
road_nodes[3][road_nodes[3].index == 8196159629]

,y,x,street_count,highway,ref,geometry,geometry_m,osmid_var
osmid,,,,,,,,
8196159629,52.397135,4.830193,1,NaN,NaN,POINT (4.83019 52.39713),POINT (624523.458 5806786.089),8196159629


In [135]:
# Block 3 population grids

grids = list()
for i in range(len(cities)):
    # extract grids
    path = 'D:/Dumps/Population_grids/' + cities[i].rsplit(',')[0] + '/' + cities[i].rsplit(',')[0] + '_CPoPGrid.gpkg'
    file = gpd.read_file(path).to_crs(4326)

    # Get grid and boundary overlay
    popgrid = file.overlay(bound_df.iloc[i:i+1])

    # Only get full grids (area differs slightly due to lon-lat placement) and grids with people in it.
    popgrid['area'] = popgrid.area / popgrid.area.max()
    popgrid = popgrid[(popgrid['area'] >= 0.99)]
    popgrid = popgrid[popgrid['PoP2015_Number'] > 0]
    
    # Get grids centroids and lon-lat
    popgrid['centroid'] = popgrid.to_crs(4326).centroid
    popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    
    popgrid = popgrid.reset_index()
    grids.append(popgrid)
grids[0]

,index,grid_id,PoP2015_Number,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,...,display_name,class,type,importance,geometry,area,centroid,centroid_m,grid_lon,grid_lat
0,6,259,113,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",0.996208,POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),-5.675978e+06,8.459813e+06
1,7,260,135,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",0.996208,POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),-5.675889e+06,8.459539e+06
2,13,400,137,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",0.996234,POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),-5.676341e+06,8.459996e+06
3,14,401,104,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01452 40.13349, -75.01452 40.131...",0.996234,POINT (-75.01564 40.13262),POINT (-5676251.742 8459722.980),-5.676252e+06,8.459723e+06
4,15,402,116,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.01227 40.13349, -75.01227 40.131...",0.996234,POINT (-75.01339 40.13262),POINT (-5676162.159 8459449.641),-5.676162e+06,8.459450e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,10279,21026,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25482 39.88135, -75.25482 39.879...",0.999949,POINT (-75.25594 39.88048),POINT (-5725941.707 8476084.513),-5.725942e+06,8.476085e+06
7869,10280,21027,22,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25257 39.88135, -75.25257 39.879...",0.999949,POINT (-75.25369 39.88048),POINT (-5725852.550 8475808.155),-5.725853e+06,8.475808e+06
7870,10281,21028,2,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25033 39.88135, -75.25033 39.879...",0.999949,POINT (-75.25145 39.88048),POINT (-5725763.377 8475531.806),-5.725763e+06,8.475532e+06
7871,10313,21166,6,40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,...,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797,"POLYGON ((-75.25931 39.87962, -75.25931 39.877...",0.999975,POINT (-75.26043 39.87875),POINT (-5726396.363 8476548.128),-5.726396e+06,8.476548e+06


In [136]:
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()
    print('extracting done')
    #parks_osm.append(gdf)
    #concat = pd.concat([gdf['osmid'],gdf['name'],gdf['element_type'],gdf['geometry']], axis = 1)
    #concat = gpd.GeoDataFrame(concat, geometry = 'geometry', crs = 4326)
    #concat.to_file('D:/Dumps/Greenspace_OSM/parks_'+i.rsplit(',')[0]+'.gpkg')
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    print(i, 'done')
    parks = parks.reset_index()
    parks_in_range.append(parks)

extracting done
Philadelphia, United States done
extracting done
Denver, United States done
extracting done
Ghent, Belgium done
extracting done
Amsterdam, Netherlands done
extracting done
Dhaka Metropolitan, Bangladesh done


In [137]:
# Block 5 park entry points

start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    #ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_proot5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Philadelphia 0.0 % done 0.03  mns
Philadelphia 11.1 % done 0.43  mns
Philadelphia 22.3 % done 0.72  mns
Philadelphia 33.4 % done 1.04  mns
Philadelphia 44.5 % done 1.36  mns
Philadelphia 55.7 % done 1.65  mns
Philadelphia 66.8 % done 1.93  mns
Philadelphia 78.0 % done 2.2  mns
Philadelphia 89.1 % done 2.48  mns
Philadelphia 100 % done 2.89  mns
Denver 0.0 % done 2.91  mns
Denver 14.7 % done 3.57  mns
Denver 29.4 % done 4.09  mns
Denver 44.1 % done 4.55  mns
Denver 58.8 % done 5.01  mns
Denver 73.5 % done 5.44  mns
Denver 88.2 % done 5.9  mns
Denver 100 % done 6.4  mns
Ghent 0.0 % done 6.4  mns
Ghent 21.3 % done 6.53  mns
Ghent 42.6 % done 6.66  mns
Ghent 63.8 % done 6.77  mns
Ghent 85.1 % done 6.89  mns
Ghent 100 % done 7.0  mns
Amsterdam 0.0 % done 7.01  mns
Amsterdam 28.6 % done 7.27  mns
Amsterdam 57.1 % done 7.54  mns
Amsterdam 85.7 % done 7.85  mns
Amsterdam 100 % done 8.07  mns
Dhaka Metropolitan 0.0 % done 8.07  mns
Dhaka Metropolitan 29.8 % done 8.19  mns
Dhaka Metropolitan 59.

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,8.466685e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14345 39.95648, -75.14348 ...",129260.205233,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,8.466609e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14638 39.95008, -75.14671 ...",152813.334108,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,8.466629e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14539 39.95528, -75.14543 ...",173200.754874,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392
3,109729590,39.955551,-75.147899,3,NaN,NaN,POINT (-75.14790 39.95555),POINT (-5709669.114 8466692.323),109729590,0,...,8.466692e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14429 39.95664, -75.14419 ...",176718.973190,766205.364911,0.230642,2.357979,1.535571,1.331006,1.187157
4,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,8.466695e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14376 39.95649, -75.14375 ...",160141.739542,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,8.464950e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5660,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,8.464937e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5661,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,8.464891e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5662,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,8.464963e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929


In [138]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5,geometry_m_buffer,components,comp_centroid,centr_dist
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178,POLYGON EMPTY,0,POINT (-75.14730 39.95683),0.0
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144,POLYGON EMPTY,1,POINT (-75.14847 39.95259),0.0
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392,POLYGON EMPTY,2,POINT (-75.14832 39.95333),0.0
3,109729590,39.955551,-75.147899,3,NaN,NaN,POINT (-75.14790 39.95555),POINT (-5709669.114 8466692.323),109729590,0,...,766205.364911,0.230642,2.357979,1.535571,1.331006,1.187157,POLYGON EMPTY,3,POINT (-75.14790 39.95555),0.0
4,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998,POLYGON EMPTY,4,POINT (-75.14763 39.95625),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5659,POINT (-75.12885 39.96700),0.0
5660,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5660,POINT (-75.12876 39.96695),0.0
5661,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5661,POINT (-75.12829 39.96718),0.0
5662,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5662,POINT (-75.12885 39.96725),0.0


In [140]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    print(cities[l])
    # Check all parks within 1000m radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/1000))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per 1000
        l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_id','grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_proot2','size_infl_proot3','size_infl_proot5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl3 = list(node_merged['size_infl_proot3'])
        infl5 = list(node_merged['size_infl_proot5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within a Gravity model variant in one of',
              thresholds,'m threshold:',len(mat_df))

        # Checks the number of the parks within 1000m.
        len_mat = len_mat + len(mat_df)

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','Grid_Id','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_proot2','size_infl_proot3','size_infl_proot5',
                      'parkshare_walked','park_area','walk_area_m2']
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_proot2','size_infl_proot3','size_infl_proot5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
        mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Philadelphia, United States
1 / 6 comb. done 0.8  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 86034
2 / 6 comb. done 1.63  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 80832
3 / 6 comb. done 2.46  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 34760
4 / 6 comb. done 3.29  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 44523
5 / 6 comb. done 4.1  mns
of 7873000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 36728
6 / 6 comb. done 4.69  mns
of 664000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 38063
total combinations within distance 320940
0.0 % gridentry done 4.72  mns
77.9 % gridentry done 5.09  mns
100 % gridentry done 5.22  mns
Denver, United States
1 / 7 comb. done 5.87  mns
of 6181000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 85130
2 / 7 comb. 

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
0,6440,1308.536022,996.379708,1091.134976,1173.387235,6440,0,14256,POINT (-75.15263 39.96348),POINT (-5708594.470 8467683.336),...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709440.723 8466685.277),129260.205233,1.313291,1.199243,1.115178,110266605,POINT (-75.15326 39.96311),POINT (-5708678.003 8467740.685),101.325
1,30059,1461.836955,951.982487,1098.294861,1231.376740,6440,3,14256,POINT (-75.15263 39.96348),POINT (-5708594.470 8467683.336),...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709669.114 8466692.323),176718.973190,1.535571,1.331006,1.187157,110266605,POINT (-75.15326 39.96311),POINT (-5708678.003 8467740.685),101.325
2,37932,1371.995350,938.581481,1065.202557,1178.692109,6440,4,14256,POINT (-75.15263 39.96348),POINT (-5708594.470 8467683.336),...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709546.213 8466695.127),160141.739542,1.461775,1.288014,1.163998,110266605,POINT (-75.15326 39.96311),POINT (-5708678.003 8467740.685),101.325
3,45805,1292.781625,961.955172,1061.560979,1148.621751,6440,5,14256,POINT (-75.15263 39.96348),POINT (-5708594.470 8467683.336),...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709444.553 8466709.353),135357.990228,1.343910,1.217812,1.125507,110266605,POINT (-75.15326 39.96311),POINT (-5708678.003 8467740.685),101.325
4,53678,1203.077755,870.704475,969.791597,1057.120582,6440,6,14256,POINT (-75.15263 39.96348),POINT (-5708594.470 8467683.336),...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709506.409 8466898.631),143083.338564,1.381729,1.240553,1.138071,110266605,POINT (-75.15326 39.96311),POINT (-5708678.003 8467740.685),101.325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320935,5194707,505.534201,510.716710,508.983330,507.600864,6400,5659,14126,POINT (-75.12568 39.96521),POINT (-5707244.779 8464470.203),...,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...",POINT (-5707086.662 8464950.373),73431.800154,0.989852,0.993223,0.995929,3449032039,POINT (-75.12743 39.96606),POINT (-5707178.169 8464727.899),266.166
320936,5202580,491.857446,496.899747,495.213262,493.868197,6400,5660,14126,POINT (-75.12568 39.96521),POINT (-5707244.779 8464470.203),...,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...",POINT (-5707090.110 8464937.109),73431.800154,0.989852,0.993223,0.995929,3449032039,POINT (-75.12743 39.96606),POINT (-5707178.169 8464727.899),266.166
320937,5210453,470.417309,475.239815,473.626845,472.340411,6400,5661,14126,POINT (-75.12568 39.96521),POINT (-5707244.779 8464470.203),...,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...",POINT (-5707034.735 8464891.123),73431.800154,0.989852,0.993223,0.995929,3449032039,POINT (-75.12743 39.96606),POINT (-5707178.169 8464727.899),266.166
320938,5218326,531.336039,536.783056,534.961207,533.508181,6400,5662,14126,POINT (-75.12568 39.96521),POINT (-5707244.779 8464470.203),...,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...",POINT (-5707046.718 8464963.244),73431.800154,0.989852,0.993223,0.995929,3449032039,POINT (-75.12743 39.96606),POINT (-5707178.169 8464727.899),266.166


In [141]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia, United States 320940
Denver, United States 316747
Ghent, Belgium 557639
Amsterdam, Netherlands 604472
Dhaka Metropolitan, Bangladesh 7799


In [226]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = 250000

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    
    # Divide in chunks of 250000 for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                except:
                    # Otherwise the nearest node is taken, which is iterated 10 times at max. Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < 25 and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(graphs[0], p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                        except:
                            try:
                                alt_route.append(nx.shortest_path(graphs[0], p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(graphs[0], g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                    except:
                                        if len3 == 10:
                                            print(i+block*k,'No route between grid and park-entry and both their 10 alternatives')
                                            pass
                                        pass
                    len4 = len(alt_route)
                    if len4 > 0: 
                        print('for index',i,'nearest node found between', 
                                               alt_route[0][0],'and',alt_route[0][-1])
                        
                        s_mat.append(alt_route[0])
                        shortest_to = list(alt_route[0][1:len(alt_route[0])])
                        shortest_to.append(0)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4,1)))
                        s_mat3.append(list(np.arange(0, len4),-1))
                        s_mat4.append('altered way')
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),' mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # get single (dissolved) line per route, attach information.
        routes2 = routes[['route','geometry']].dissolve('route')
        routes2['way_calculated'] = s_mat4
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['num_steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid',
                                              'grid_osm','walk_area_m2','size_infl_proot2','size_infl_proot3',
                                              'size_infl_proot5','raw euclidean']],
                                                
                            left_index = True, right_index = True)
        routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
        routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
        routes2['grav3_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot3']
        routes2['grav5_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

Amsterdam
1 / 3 range 0 - 250000
0.0 % done 0.01 mns
1.65 % done 0.78 mns
3.31 % done 1.34 mns
4.96 % done 1.78 mns
6.62 % done 2.33 mns
8.27 % done 2.84 mns
9.93 % done 3.37 mns
11.58 % done 3.96 mns
13.23 % done 4.59 mns
14.89 % done 5.06 mns
16.54 % done 5.75 mns
18.2 % done 6.52 mns
19.85 % done 7.02 mns
21.51 % done 7.71 mns
23.16 % done 8.06 mns
24.82 % done 8.62 mns
26.47 % done 9.08 mns
28.12 % done 9.5 mns
29.78 % done 10.04 mns
31.43 % done 10.6 mns
33.09 % done 11.56 mns
34.74 % done 12.26 mns
36.4 % done 12.89 mns
38.05 % done 13.46 mns
39.7 % done 14.29 mns
41.36 % pathfinding done 14.82  mns
formatting done 17.98 mns
dissolving done 19.5 mns
2 / 3 range 250000 - 500000
41.36 % done 19.51 mns
43.01 % done 20.21 mns
44.67 % done 20.69 mns
46.32 % done 21.41 mns
47.98 % done 22.06 mns
49.63 % done 22.82 mns
51.28 % done 23.35 mns
52.94 % done 23.82 mns
54.59 % done 24.55 mns
56.25 % done 25.18 mns
57.9 % done 25.69 mns
59.56 % done 26.28 mns
61.21 % done 26.66 mns
62.86 % do

,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,raw euclidean,raw_total_cost,grav2_total_cost,grav3_total_cost,grav5_total_cost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((4.91113 52.36878, 4.91104 52...",normal way,2496.013,44,4199,0,0,22.709,46326678,287088914,124243.391058,1.251999,1.161634,1.094061,1247.236560,2518.722,2011.760786,2168.258067,2302.176819,4199-0
1,"MULTILINESTRING ((4.91113 52.36878, 4.91104 52...",normal way,2342.618,35,4199,0,2,22.709,46341640,287088914,119552.804274,1.228138,1.146827,1.085673,1201.299551,2365.327,1925.945780,2062.495932,2178.674429,4199-0
2,"MULTILINESTRING ((4.91113 52.36878, 4.91104 52...",normal way,2291.916,30,4199,0,3,22.709,46344112,287088914,116747.464200,1.213643,1.137786,1.080529,1148.436678,2314.625,1907.171123,2034.323370,2142.122233,4199-0
3,"MULTILINESTRING ((4.91113 52.36878, 4.91104 52...",normal way,1812.149,32,4199,0,4,22.709,46344541,287088914,121028.393648,1.235694,1.151526,1.088339,1139.637571,1834.858,1484.880747,1593.413665,1685.924439,4199-0
4,"MULTILINESTRING ((4.91113 52.36878, 4.91104 52...",normal way,1840.023,34,4199,0,5,22.709,90736244,287088914,122559.847719,1.243487,1.156363,1.091080,1163.736096,1862.732,1497.990365,1610.853847,1707.237074,4199-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604467,"MULTILINESTRING ((4.76092 52.35525, 4.76080 52...",normal way,835.766,6,5040,174,4042,107.802,413575563,1764586019,233224.302917,1.715356,1.432967,1.240914,545.460907,943.568,550.071385,658.471683,760.381503,5040-174
604468,"MULTILINESTRING ((4.76092 52.35525, 4.76080 52...",normal way,563.914,5,5040,174,4043,107.802,1764586151,1764586019,155400.504087,1.400211,1.251591,1.144135,254.295342,671.716,479.724869,536.689855,587.094902,5040-174
604469,"MULTILINESTRING ((4.76092 52.35525, 4.76080 52...",normal way,968.415,5,5040,174,4044,107.802,1766754967,1764586019,214850.919474,1.646402,1.394303,1.220715,606.251041,1076.217,653.678183,771.867242,881.628265,5040-174


In [227]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['PoP2015_Number','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['PoP2015_Number']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['PoP2015_Number'] / thold['walk_area_ha' + str2]
            #thold = thold[thold[score] > 0]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + score,'walkha_person_' + score]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2, 'walkha_person' + str2].sum()
            
            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['PoP2015_Number'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            gs.to_file('D:Dumps/Scores output OSM/Grid_lines_shp/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['PoP2015_Number'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['PoP2015_Number'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        gridsc.to_file('D:Dumps/Scores output OSM/Grid_geoms_shp/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        gridsc.to_csv('D:/Dumps/Scores output OSM/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities[

popg_acc.to_csv('D:/Dumps/Scores output OSM/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output OSM/radius-euclidean adjusted popgrid access.csv')

adj_popg_acc    


Amsterdam, Netherlands
entrance 1.03 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 1.44 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 1.82 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 2.17 mns
grid  300
grid  600
grid  1000
Philadelphia, United States done 2.47 mns
Dhaka Metropolitan, Bangladesh
entrance 2.48 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 2.66 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 2.84 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 3.02 mns
grid  300
grid  600
grid  1000
Denver, United States done 3.2 mns


Amsterdam, Netherlands  \
entrance_300        1 high                  0.136936   
                    2 medium                0.063317   
                    3 low                   0.070516   
                    4 no                    0.729231   
entrance_600        1 high                  0.079079   
                    2 medium                0.222030   
                    3 low                   0.229685   
                    4 no                    0.469206   
entrance_1000       1 high                  0.101741   
                    2 medium                0.258882   
                    3 low                   0.353999   
                    4 no                    0.285377   
gravity**(1/2)_300  1 high                  0.140714   
                    2 medium                0.066332   
                    3 low                   0.074195   
                    4 no                    0.718759   
gravity**(1/2)_600  1 high                  0.047054   
                    2 medium                0.258599   
                    3 low                   0.201263   
                    4 no                    0.493084   
gravity**(1/2)_1000 1 high                  0.033112   
                    2 medium                0.284324   
                    3 low                   0.390616   
                    4 no                    0.291948   
gravity**(1/3)_300  1 high                  0.128388   
                    2 medium                0.070111   
                    3 low                   0.065296   
                    4 no                    0.736205   
gravity**(1/3)_600  1 high                  0.040401   
                    2 medium                0.244266   
                    3 low                   0.206832   
                    4 no                    0.508502   
gravity**(1/3)_1000 1 high                  0.030206   
                    2 medium                0.254839   
                    3 low                   0.401085   
                    4 no                    0.313869   
gravity**(1/5)_300  1 high                  0.127315   
                    2 medium                0.066355   
                    3 low                   0.062131   
                    4 no                    0.744199   
gravity**(1/5)_600  1 high                  0.046869   
                    2 medium                0.232406   
                    3 low                   0.215590   
                    4 no                    0.505135   
gravity**(1/5)_1000 1 high                  0.036273   
                    2 medium                0.253169   
                    3 low                   0.393186   
                    4 no                    0.317373   

                              Dhaka Metropolitan, Bangladesh  
entrance_300        1 high                          0.014018  
                    2 medium                        0.018168  
                    3 low                           0.020494  
                    4 no                            0.947320  
entrance_600        1 high                          0.008742  
                    2 medium                        0.047638  
                    3 low                           0.110310  
                    4 no                            0.833311  
entrance_1000       1 high                          0.004495  
                    2 medium                        0.044752  
                    3 low                           0.227392  
                    4 no                            0.723361  
gravity**(1/2)_300  1 high                          0.032992  
                    2 medium                        0.027545  
                    3 low                           0.033300  
                    4 no                            0.906163  
gravity**(1/2)_600  1 high                          0.018421  
                    2 medium                        0.089245  
                    3 low                           0.164589  
                    4 no 

In [228]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'walkha_' + score]] = str1.groupby(
                'Park_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            #prk = prk.fillna(-1)
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
            prk.to_file('D:Dumps/Scores output OSM/Park_lines_shp/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        prksc.to_file('D:Dumps/Scores output OSM/Grid_geoms_shp/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        prksc.to_csv('D:/Dumps/Scores output OSM/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[1][1])

park 300
park 600
park 1000
entrance 0.31 mns
park 300
park 600
park 1000
gravity**(1/2) 0.54 mns
park 300
park 600
park 1000
gravity**(1/3) 0.63 mns
park 300
park 600
park 1000
gravity**(1/5) 0.7 mns
Amsterdam, Netherlands done 0.7 mns
park 300
park 600
park 1000
entrance 0.72 mns
park 300
park 600
park 1000
gravity**(1/2) 0.76 mns
park 300
park 600
park 1000
gravity**(1/3) 0.78 mns
park 300
park 600
park 1000
gravity**(1/5) 0.79 mns
Dhaka Metropolitan, Bangladesh done 0.79 mns


,tr_300,pop_tr_300,walkha_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,walkha_tr_600,...,tr_600route,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,walkha_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,93.352951,2.019224e+05,4.632626,1.0,"(LINESTRING (90.4196392 23.8054347, 90.4194595...",[1287]:[0],3 low,1010.828471,2.072572e+06,27.795757,...,"(LINESTRING (90.417658 23.8073537, 90.4177306 ...","[1243, 1244, 1286, 1287, 1330, 1331]:[0, 0, 0,...",1 high,4252.874289,8.238578e+06,41.693636,9.0,"(LINESTRING (90.4200932 23.8090468, 90.4197327...","[1200, 1243, 1244, 1286, 1287, 1288, 1329, 133...",1 high
1,1452.991679,2.897260e+06,298.296854,14.0,"(LINESTRING (90.4154558 23.8032791, 90.4154951...","[1328, 1371, 1372, 1412, 1413, 1414, 1415, 145...",1 high,9515.269663,1.911544e+07,937.466238,...,"(LINESTRING (90.415171 23.8053493, 90.4138805 ...","[1285, 1324, 1325, 1326, 1327, 1328, 1367, 136...",1 high,30320.645361,5.890364e+07,1265.540532,58.0,"(LINESTRING (90.4080935 23.8049378, 90.4080779...","[1282, 1283, 1284, 1285, 1286, 1287, 1324, 132...",1 high
2,1029.720183,1.674642e+06,194.879084,13.0,"(LINESTRING (90.4080935 23.8049378, 90.4080779...","[1282, 1324, 1325, 1326, 1327, 1367, 1368, 136...",1 high,7928.384685,1.453272e+07,597.209778,...,"(LINESTRING (90.4080935 23.8049378, 90.4080779...","[1239, 1280, 1281, 1282, 1285, 1323, 1324, 132...",1 high,24928.842227,4.566266e+07,899.893602,49.0,"(LINESTRING (90.4046381 23.810176, 90.4045951 ...","[1194, 1196, 1197, 1237, 1238, 1239, 1241, 124...",1 high
3,842.274518,1.480345e+06,32.608086,5.0,"(LINESTRING (90.4196392 23.8054347, 90.4177607...","[1286, 1287, 1329, 1416, 1460]:[19, 18, 22, 14...",2 medium,4061.823822,7.596279e+06,121.115100,...,"(LINESTRING (90.4192166 23.8072591, 90.4193298...","[1244, 1286, 1287, 1288, 1329, 1330, 1331, 137...",1 high,14975.352358,2.831127e+07,303.078634,35.0,"(LINESTRING (90.417658 23.8073537, 90.4177306 ...","[1243, 1244, 1286, 1287, 1288, 1328, 1329, 133...",1 high
4,695.538423,1.511976e+06,45.493615,5.0,"(LINESTRING (90.4150593 23.7932749, 90.4155104...","[1548, 1549, 1593, 1594, 1595]:[30, 34, 32, 32...",2 medium,4614.019513,9.857725e+06,181.974458,...,"(LINESTRING (90.4130916 23.7955013, 90.4130047...","[1502, 1503, 1504, 1505, 1547, 1548, 1549, 155...",1 high,14191.067568,2.975880e+07,254.764242,28.0,"(LINESTRING (90.4130454 23.7974731, 90.4131026...","[1457, 1458, 1459, 1501, 1502, 1503, 1504, 150...",1 high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,150.248633,3.694614e+05,2.916848,1.0,"(LINESTRING (90.413408 23.7724674, 90.4134713 ...",[1995]:[329],3 low,924.487336,2.474598e+06,14.584240,...,"(LINESTRING (90.4130678 23.7744734, 90.4128941...","[1952, 1994, 1995, 2037, 2038]:[329, 329, 329,...",2 medium,3208.930749,8.453280e+06,17.501088,6.0,"(LINESTRING (90.4109673 23.774287, 90.4110237 ...","[1951, 1952, 1994, 1995, 2037, 2038]:[329, 329...",1 high
164,263.994178,4.243547e+05,7.827551,2.0,"LINESTRING (90.388625 23.8716125, 90.3878307 2...","[51, 52]:[334, 336]",3 low,1364.946258,2.028528e+06,27.396427,...,"(LINESTRING (90.3839216 23.8737428, 90.3839299...","[12, 13, 14, 51, 52, 86, 87]:[338, 338, 338, 3...",1 high,5848.626848,8.378738e+06,46.965304,12.0,"(LINESTRING (90.3839216 23.8737428, 90.3839299...","[12, 13, 14, 15, 50, 51, 52, 53, 86, 87, 88, 8...",1 high
165,NaN,NaN,NaN,NaN,NaN,NaN,0,171.416680,3.154067e+04,0.309595,...,"LINESTRING (90.359232 23.8448127, 90.3591054 2...",[456]:[339],3 low,968.160688,2.011527e+05,0.619191,2.0,"(LINESTRING (90.3570749 23.8446331, 90.3570759...","[455, 456]:[339, 339]",2 medium
166,736.086768,3.167238e+06,63.512045,6.0,"(LINESTRING (90.4308054 23.7413913, 90.4308091...","[2653, 2691, 2692, 2693, 2729, 2730]:[341, 343...",2 medium,4839.224393,2.131580e+07,222.292156,...,"(LINESTRING (90.429054 23.7433503, 90.4290628 ...","[2612, 2613, 2651, 2652, 2653, 2654, 2655, 269...",1 high,14424.00

In [ ]:
grid_scores[0][0][grid_scores[0][0]['tr_300']>0]

In [229]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['grid_id'], how = 'outer')
            g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')].to_csv('D:/Dumps/Scores output OSM/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            g_lines.to_file('D:Dumps/Scores output OSM/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            g_geoms.to_file('D:Dumps/Scores output OSM/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')

Amsterdam, Netherlands
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
Dhaka Metropolitan, Bangladesh
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
all done


In [ ]:
print(round((time.time() - start) / 60,2),'mns')